# Reservas Provadas de Petróleo e Gás Natural

In [ ]:
import pandas as pd
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.2")
df.drop(df.head(4).index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(df.tail(12).index, inplace=True)
df.drop(columns=df.columns[0], axis=1, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ANO"
df = df[1:]
df.columns = new_header
df.dropna(inplace=True)
df.rename(columns={
    'ANO': 'ano',
    'PETRÓLEO': 'petróleo_10^3m3',
    'GÁS NATURAL': 'gás_natural_10^6m3'
}, inplace=True)
df

# Recursos Hidráulicos

In [ ]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.3")
df.drop(df.head(5).index, inplace=True)
df.drop(df.tail(1).index, inplace=True)
df.drop(columns=df.columns[0], axis=1, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ano"
df = df[1:]
df.columns = new_header
df.rename(columns = {
    'INVENTARIADO + APROVEITADO / INVENTORIED': "inventariado_mw",
    'ESTIMADO / ESTIMATED\n': "estimado_mw",
    'TOTAL': 'total_mw'
}, inplace=True)
df.reset_index(inplace=True, drop=True)
df.dropna(inplace=True)
df

# Reservas de Carvão Mineral

In [ ]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.4")
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.drop(df.head(5).index, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ano"
df = df[1:]
df.columns = new_header
df.drop(df.head(1).index, inplace=True)
df.rename(columns=lambda x: x.lower(), inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

# Reservas de Urânio

In [ ]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.5")
df.drop(df.head(6).index, inplace=True)
df.drop(columns=df.columns[0], inplace=True)
new_header = df.iloc[0]
df = df[1:]
new_header.iloc[0] = "ano"
df.columns = new_header
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.rename(columns = {
    'U3O8': "ton_U3O8"
})
df

# Recursos e Reservas energéticas brasileiras

In [128]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="Tabela 6.1")
df.drop(df.head(3).index, inplace=True)
df.dropna(how='all', inplace=True, axis=0)
df.reset_index(inplace=True,drop=True)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.drop(index=df.index[6:23], inplace=True)
df.drop(df.tail(6).index, inplace=True)
df.drop(df.head(2).index, inplace=True)
df = df.iloc[:, :-2]
df.dropna(inplace=True)
df = df.set_axis(['energia', 'unidade_de_medida', 'medidas', 'inferidas', 'total', 'equivalência'], axis=1)
df['energia'] = df['energia'].apply(lambda x: x.split(' ')[0])
df.reset_index(inplace=True, drop=True)
df

,energia,unidade_de_medida,medidas,inferidas,total,equivalência
0,CARVÃO,106 t,25695.49424,6535,32230.49424,7014869.927499
1,HIDRÁULICA,GW,111.442485,23.989917,135.432401,83956.3105
2,ENERGIA,t U3O8,177500,131870,309370,2154010.622349
